In [27]:
import pandas as pd 
import numpy as np 

Reading data

In [28]:
data_path = '../Datasets/Data23.xlsx'

sales = pd.read_excel(data_path,sheet_name = 'SALES')
products = pd.read_excel(data_path,sheet_name = "PRODUCTS")
customers = pd.read_excel(data_path,sheet_name = 'CUSTOMERS')

In [29]:
sales.head(3)

,TransactionID,UserID,DateTime,ProductID,Channel,PaymentType,Price,Discount
0,1,500546547,2017-01-01 01:40:39.180,10334,MOBILE,Cash,51.0,No
1,2,500240813,2017-01-01 03:27:58.490,10610,WEB,Cash,108.0,No
2,3,500460527,2017-01-01 04:42:46.500,10579,MOBILE,Cash,22.5,No


In [30]:
products.head(3)

,ProductID,Category
0,10001,Female Shoes
1,10002,Male Shoes
2,10003,TVs and TV Sets


In [31]:
customers.head(3)

,UserID,UserFirstTransaction,Gender,Location,Age
0,500234532,2011-10-12 00:00:00.000,FEMALE,ANTALYA,19
1,500234631,2018-04-06 15:19:33.990,FEMALE,BURSA,20
2,500234642,2016-04-23 00:00:00.000,MALE,IZMIR,41


Bring all together

In [32]:
df = pd.merge(
    pd.merge(sales,products, on = 'ProductID'),
    customers, on = 'UserID')
df.head()

,TransactionID,UserID,DateTime,ProductID,Channel,PaymentType,Price,Discount,Category,UserFirstTransaction,Gender,Location,Age
0,1,500546547,2017-01-01 01:40:39.180,10334,MOBILE,Cash,51.0,No,Female Shoes,2015-03-18,FEMALE,ANKARA,30
1,8317,500546547,2017-03-30 23:53:07.050,10334,MOBILE,Cash,21.0,No,Female Shoes,2015-03-18,FEMALE,ANKARA,30
2,45056,500546547,2018-02-03 18:32:01.840,10173,MOBILE,Mobile Payment,66.0,No,Sport Shoes,2015-03-18,FEMALE,ANKARA,30
3,47085,500546547,2018-02-17 18:03:09.930,10173,MOBILE,Mobile Payment,96.0,No,Sport Shoes,2015-03-18,FEMALE,ANKARA,30
4,52212,500546547,2018-03-22 12:04:56.920,10173,MOBILE,Cash,45.0,No,Sport Shoes,2015-03-18,FEMALE,ANKARA,30


### Churn Değişkeni

Müşterinin son 1 yıl içerisinde hiç alışveriş yapmamış olmasını churn etmiş olarak kabul edip buna göre churn değişkeni oluşturuyorum.  

* UserID'ye göre gruplandırıp her kullanıcının en son alışveriş tarihlerini al. Son alışveriş tarihinden x yıl içerisinde işlem yapmış mı?
* Verilerin çekildiği tarihi bilmediğimiz için en son yapılan işlem tarihini (maksimum tarih) verilerin alındığı tarih (O an) kabul edeceğim. 
* Churn değişkenini oluştur

In [33]:
last_transaction_dates = df.groupby('UserID')['DateTime'].max().reset_index()
last_transaction_dates.columns = ['UserID', 'LastTransactionDate']
last_transaction_dates.head(3)

,UserID,LastTransactionDate
0,500234532,2017-12-17 18:08:28.150
1,500234631,2018-05-11 13:22:46.950
2,500234642,2018-06-17 19:15:22.400


In [34]:
df = pd.merge(df, last_transaction_dates, on='UserID')
df['Now'] = df['DateTime'].max()
df.head(3)

,TransactionID,UserID,DateTime,ProductID,Channel,PaymentType,Price,Discount,Category,UserFirstTransaction,Gender,Location,Age,LastTransactionDate,Now
0,1,500546547,2017-01-01 01:40:39.180,10334,MOBILE,Cash,51.0,No,Female Shoes,2015-03-18,FEMALE,ANKARA,30,2018-06-28 19:35:53.360,2018-06-30 23:56:14.570
1,8317,500546547,2017-03-30 23:53:07.050,10334,MOBILE,Cash,21.0,No,Female Shoes,2015-03-18,FEMALE,ANKARA,30,2018-06-28 19:35:53.360,2018-06-30 23:56:14.570
2,45056,500546547,2018-02-03 18:32:01.840,10173,MOBILE,Mobile Payment,66.0,No,Sport Shoes,2015-03-18,FEMALE,ANKARA,30,2018-06-28 19:35:53.360,2018-06-30 23:56:14.570


* Müşterilerin en fazla kaç gündür işlem yapmadığını bulalım. Ve buna göre müşterilerin kaç günden fazla işlem yapmazsa churn kabul edileceğini belirleyelim.

In [35]:
last_transaction_dates['Now'] = df['DateTime'].max()
last_transaction_dates['Recency'] = (last_transaction_dates['Now'] - last_transaction_dates['LastTransactionDate']).dt.days
last_transaction_dates.head(3)

,UserID,LastTransactionDate,Now,Recency
0,500234532,2017-12-17 18:08:28.150,2018-06-30 23:56:14.570,195
1,500234631,2018-05-11 13:22:46.950,2018-06-30 23:56:14.570,50
2,500234642,2018-06-17 19:15:22.400,2018-06-30 23:56:14.570,13


In [36]:
import plotly.express as px

fig = px.box(last_transaction_dates, x='Recency')
fig.show()

In [37]:
last_transaction_dates['Recency'].describe()

count    3578.000000
mean      138.520123
std       153.303295
min         0.000000
25%        11.000000
50%        67.500000
75%       252.000000
max       545.000000
Name: Recency, dtype: float64

* Sağa çarpık bir dağılım gösteriyor müşterilerin en son ne zaman alış-veriş yaptığının dağılımı. Ortalama < Medyan  
* Çoğu müşteri ortalama olarak sık alışveriş yapıyor. Bu nedenle mod ya da ortalama yerine medyan değeri ele almak istiyorum.  
* Ancak eğer medyan değeri referans alırsam verilerim çok fazla dengesiz olacak. Bu nedenle bu modele mahsus birazcık daha tolere davranarak 3. çeyrekiği ele alıyorum. Bu sayede churn eden müşteri sayısı fazla olmamış olur ki asıl bu dengesiz bir şirketi ortaya çıkarır.    
* Müşterilerin churned kabul edilmesi konusunda biraz daha insaflı davranan bir şirket olduğumuzu varsayalım.

In [38]:
df['Churn'] = (pd.to_datetime(df['Now']) - pd.to_datetime(df['LastTransactionDate'])).dt.days > 252
df['Churn'] = df['Churn'].astype(int)
df.head(3)

,TransactionID,UserID,DateTime,ProductID,Channel,PaymentType,Price,Discount,Category,UserFirstTransaction,Gender,Location,Age,LastTransactionDate,Now,Churn
0,1,500546547,2017-01-01 01:40:39.180,10334,MOBILE,Cash,51.0,No,Female Shoes,2015-03-18,FEMALE,ANKARA,30,2018-06-28 19:35:53.360,2018-06-30 23:56:14.570,0
1,8317,500546547,2017-03-30 23:53:07.050,10334,MOBILE,Cash,21.0,No,Female Shoes,2015-03-18,FEMALE,ANKARA,30,2018-06-28 19:35:53.360,2018-06-30 23:56:14.570,0
2,45056,500546547,2018-02-03 18:32:01.840,10173,MOBILE,Mobile Payment,66.0,No,Sport Shoes,2015-03-18,FEMALE,ANKARA,30,2018-06-28 19:35:53.360,2018-06-30 23:56:14.570,0


* Hedef değişkenimizi oluşturduğumuza göre asıl modelimizi belirlemek adına `LastTransactionDate` ve `Now` değişkenlerini kaldıracağım. Çünkü modeli kurup kullanılabilir hale getirdiğimizde kullanıcının son siparişinin üstünden belirlenen threshold günü geçmeden onun churn edip etmeyeceğini bulmalıyız.  

* Ayrıca müşterilerin yaptığı her alış-verişi modele dahil etmemeliyiz. Müşteri üzerinde genel bir yargımız olmalı.  
* Müşterimiz herhangi bir şey aldığında model her seferinde onu değerlendirmemeli genel manada ele almalı.  

In [39]:
df.drop(['Now','LastTransactionDate'], axis=1, inplace=True)

* Hedef değişken dengesi

In [40]:
df['Churn'].value_counts()

0    64246
1     4813
Name: Churn, dtype: int64